In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


engine = create_engine("sqlite:///regression_database.db")

chrome_options = Options()
chrome_options.add_argument("user-data-dir=selenium") 

sql = """select DISTINCT *
FROM
 (
     select book_title
    from goodreads_best_book_titles
     union ALL
    select book_title
     from goodreads_worst_book_titles
 );"""

df_books = pd.read_sql(sql, engine)



In [ ]:
### Setting up definitions

def genere_pull(tag, class_search): #soup.find_all('a', class_="actionLinkLite bookPageGenreLink")
    test = soup.find(tag, class_=class_search)
    if test is not None:
        for i, x in enumerate(soup.find_all(tag, class_=class_search), 1):
            if i == 1:
                genre_1 = x.text
    else:
        genre_1 = None 
    return genre_1


def char_counter(tag, class_search, id_search): #soup.find('div', class_="uitext", id="bookDataBox").find_all('a')
    char_count = 0
    if soup.find(tag, class_=class_search, id=id_search) is not None:
        for x in soup.find(tag, class_=class_search, id=id_search).find_all('a'):
            if '/characters/' in x['href']:
                char_count += 1
            else:
                continue
    else:
        char_count = 0
    return str(char_count)


def series_y_n(soup_series):
    return soup_series != None

def awards_y_n(soup_awards):
    return soup_awards != None


def rating_reveiws(tag, class_search, href_search): #soup.find_all('a', class_="gr-hyperlink", href='#other_reviews')
    if soup.find(tag, class_=class_search, href=href_search) is not None:
        ratings_reviews = [x.text.replace('\n', "").strip() for x in soup.find_all(tag, class_=class_search, href=href_search)]
        for i, x in enumerate(ratings_reviews):
            if i == 0:
                ratings = x
            elif i == 1:
                reviews = x
            else:
                continue
    else:
        ratings = "no rate"
        reviews = "no review"
    return ratings.replace(" ratings", "").strip(), reviews.replace(" reviews", "").strip()




In [ ]:
base_url = 'https://www.goodreads.com'


loop_counter = 1

while len(df_books['book_title']) > loop_counter:
    for i, x in enumerate(df_books['book_title']):
        try:
            book_url = base_url+x
            response = requests.get(book_url)
            page = response.text
            soup = BeautifulSoup(page, 'html.parser')
            book_cover_type = soup.find('div', id="details", class_="uitext").find_all('span')[:3][0].text
            num_of_pages = soup.find('span', itemprop="numberOfPages").text.replace(" pages", "").strip()
            book_user_rating = soup.find('span', itemprop="ratingValue").text.replace('\n', "").strip()
            title = soup.find('h1').text.replace('\n', "").strip()
            language = soup.find('div', class_='infoBoxRowItem', itemprop="inLanguage").text
            series = series_y_n(soup.find('div', class_="infoBoxRowTitle", text ="Series"))
            awards = series_y_n(soup.find('div', class_="infoBoxRowTitle", text ="Literary Awards"))
            char_counters = char_counter('div', "uitext", "bookDataBox")
            user_rate, written_reviews = rating_reveiws('a', "gr-hyperlink", '#other_reviews')
            genre_one, genre_two, genre_three = genere_pull('a', "actionLinkLite bookPageGenreLink")


            headers = ['book_cover_type', 
                       'num_of_pages', 'book_user_rating',
                      'title', 'language', 'is_series', 'won_awards', 'num_of_chars', 
                       'num_user_rate', 'num_written_reviews', 'genre_1', 'genre_2', 'genre_3']

            book_dict = dict(zip(headers, [book_cover_type,
                                            num_of_pages, 
                                            book_user_rating,
                                           title,
                                           language,
                                           series,
                                           awards,
                                          char_counters,
                                          user_rate.replace(",", ""),
                                          written_reviews.replace(",", ""),
                                          genre_one,
                                          genre_two,
                                          genre_three]))
            book_data.append(book_dict)
            loop_counter += 1
        except AttributeError:
            print(f"{x}, {i} did not pass test")
            continue
        print(f"{x}, {i} title completed")


In [ ]:
book_data
df_book_data_output = pd.DataFrame(book_data)  #convert list of dict to df


In [ ]:
df_book_data_output.head()

In [ ]:
### save to SQLLite DB to save information

from sqlalchemy import create_engine
engine = create_engine("sqlite:///regression_database.db")
sqlite_connection = engine.connect()
sqlite_table = "tbl_book_data_output"
df_book_data_output.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

In [ ]:
### Save to CSV for good measures
df_book_data_output.to_csv('goodreads_webscrapping_output.csv')